<a href="https://colab.research.google.com/github/thainazanfolin/min_sleep_efficiency/blob/main/Projeto_Min_Sleep_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Projeto de Análise de dados: como os fatores de Estilo de Vida influenciam a Eficiência do Sono

- [Link do dataset no Kaggle](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency/data)

## Objetivo do projeto
Realizar uma análise sobre como diferentes hábitos podem ou não afetar a eficiência do sono, considerando tempo de sono e também sua qualidade.

## Sobre o Conjunto de Dados
O conjunto de dados contém informações sobre um grupo de sujeitos de teste e seus padrões de sono.
- **Subject ID**: Identificação exclusiva de cada sujeito, com idade e gênero registrados.
- **Bedtime**: Hora em que cada sujeito vai para a cama.
- **Wakeup time**: Hora em que cada sujeito acorda.
- **Sleep duration**: Total de tempo dormido em horas.
- **Sleep efficiency**: Proporção do tempo na cama que foi realmente gasto dormindo.
- **REM sleep percentage**: Tempo gasto em sono REM.
- **Deep sleep percentage**: Tempo gasto em sono profundo.
- **Light sleep percentage**: Tempo gasto em sono leve.
- **Awakenings**: Número de vezes que cada sujeito acorda durante a noite.
- **Caffeine consumption**: Consumo de cafeína nas 24 horas anteriores à hora de dormir.
- **Alcohol consumption**: Consumo de álcool nas 24 horas anteriores à hora de dormir.
- **Smoking status**: Estado de fumante de cada sujeito.
- **Exercise frequency**: Frequência de exercícios de cada sujeito.



## Vamos importar a nossa base de dados.
 Optamos por trazer por URL para facilitar o trabalho de todos os membros da equipe em um só colab. <br> Porém, também pode ser feito subindo a base aqui no Colab, normalmente. <br><br>
Vamos começar!

In [3]:
#importando as bibliotecas necessárias
import pandas as pd
import time


In [4]:
#trazendo o database de uma url externa

url = 'https://raw.githubusercontent.com/thainazanfolin/min_sleep_efficiency/refs/heads/main/Sleep_Efficiency_table.csv'
df = pd.read_csv(url, delimiter=';')

# Verifique as primeiras linhas do DataFrame
print(df.head(100))


     ID  Age  Gender           Bedtime       Wakeup time  Sleep duration  \
0     1   65  Female  06/03/2021 01:00  06/03/2021 07:00             6.0   
1     2   69    Male  05/12/2021 02:00  05/12/2021 09:00             7.0   
2     3   40  Female  25/05/2021 21:30  25/05/2021 05:30             8.0   
3     4   40  Female  03/11/2021 02:30  03/11/2021 08:30             6.0   
4     5   57    Male  13/03/2021 01:00  13/03/2021 09:00             8.0   
..  ...  ...     ...               ...               ...             ...   
95   96   61  Female  10/07/2021 02:00  10/07/2021 09:00             7.0   
96   97   41    Male  15/02/2021 21:00  15/02/2021 04:00             7.0   
97   98   53    Male  02/11/2021 23:00  02/11/2021 06:30             7.5   
98   99   32  Female  12/01/2021 02:00  12/01/2021 11:00             9.0   
99  100   65    Male  07/12/2021 01:30  07/12/2021 09:30             8.0   

    Sleep efficiency  REM sleep percentage  Deep sleep percentage  \
0               0.

In [7]:
# Observando as informações gerais sobre o dataset, incluindo tipos de dados e valores nulos
df.info()

# Verificar quantos valores nulos existem em cada coluna
df.isnull().sum()

print('-------')

# Verificar se há linhas duplicadas
linhas_duplicadas = df.duplicated().sum()
print(f"Número de linhas duplicadas: {linhas_duplicadas}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      452 non-null    int64  
 1   Age                     452 non-null    int64  
 2   Gender                  452 non-null    object 
 3   Bedtime                 452 non-null    object 
 4   Wakeup time             452 non-null    object 
 5   Sleep duration          452 non-null    float64
 6   Sleep efficiency        452 non-null    float64
 7   REM sleep percentage    452 non-null    int64  
 8   Deep sleep percentage   452 non-null    int64  
 9   Light sleep percentage  452 non-null    int64  
 10  Awakenings              432 non-null    float64
 11  Caffeine consumption    427 non-null    float64
 12  Alcohol consumption     438 non-null    float64
 13  Smoking status          452 non-null    object 
 14  Exercise frequency      446 non-null    fl

A partir das informações da tabela conseguimos verificar que:

* existem **452 registros** totais
* nas colunas 'Awakenings', 'Caffeine consumption', 'Alcohol consumption' e 'Exercise frequency' **existem dados faltantes (nulos)**
* não existem linhas duplicadas   

Vamos trazer mais dados sobre os valores nulos para decidir como vamos fazer a limpeza dos dados.

In [10]:
#vamos ver quantos nulos temos em cada coluna

null_awakenings = df['Awakenings'].isnull().sum()
print(f"Número de nulos em Awakenings: {null_awakenings}")

null_caffeine = df['Caffeine consumption'].isnull().sum()
print(f"Número de nulos em Caffeine consumption: {null_caffeine}")

null_alcohol = df['Alcohol consumption'].isnull().sum()
print(f"Número de nulos em Alcohol consumption: {null_alcohol}")

null_exercise = df['Exercise frequency'].isnull().sum()
print(f"Número de nulos em Exercise frequency: {null_exercise}")


# Total de nulos no dataframe
sum_null = sum([null_awakenings, null_caffeine, null_alcohol, null_exercise])
print(f"Total de nulos no dataframe: {sum_null}")


Número de nulos em Awakenings: 20
Número de nulos em Caffeine consumption: 25
Número de nulos em Alcohol consumption: 14
Número de nulos em Exercise frequency: 6
Total de nulos no dataframe: 65


Como vimos, temos o total de 65 valores em branco no dataframe. Nossas opções para limpar esses dados são:

1. excluir as linhas com dados faltantes
2. preencher os dados faltantes com a média dos outros valores (verificar se faz sentido)
3. preencher os dados faltantes com o dado presente em outro caso semelhante (podemos fazer um order by por idade e genero, por exemplo, e trazer o mesmo dado).

Considerando que nossa database tem poucos registros, vamos fazer a opção 1 e 2, para não perder registros. Porém, vamos verificar o dado faltante de acordo com a coluna. <br><br>

Vamos retomar as informações da database, considerando o que nossas colunas trazem:
* Awakenings: Número de vezes que cada sujeito acorda durante a noite
<br>**(é possível fazer uma média entre todos e preencher)** <br><br>

* Caffeine consumption: Consumo de cafeína nas 24 horas anteriores à hora de dormir.
<br>**(é possível fazer uma média entre todos e preencher, porém é importante verificar antes a quantidade de ZEROS presentes, pois a média pode não representar a realidade. Interessante talvez saber se a maioria é 0, se sim, podemos preencher com zeros)** <br><br>

* Alcohol consumption: Consumo de álcool nas 24 horas anteriores à hora de dormir. <br>**(é possível fazer uma média entre todos e preencher, porém é importante verificar antes a quantidade de ZEROS presentes, pois a média pode não representar a realidade. Interessante talvez saber se a maioria é 0, se sim, podemos preencher com zeros)**<br><br>

* Exercise frequency: Frequência de exercícios de cada sujeito.
<br>**(é possível fazer uma média entre todos e preencher)** <br><br>




In [ ]:
# todo: fazer aqui o que está acima em blocos separados

# BLOCO Awakenings: é possível fazer uma média entre todos e preencher


In [ ]:
# BLOCO Caffeine consumption: é possível fazer uma média entre todos e preencher, porém é importante verificar antes a quantidade de ZEROS presentes, pois a média pode não representar a realidade. Interessante talvez saber se a maioria é 0, se sim, podemos preencher com zeros
# todo: somar a quantidade de zeros nessa coluna para avaliar

In [ ]:
# BLOCO Alcohol consumption: é possível fazer uma média entre todos e preencher, porém é importante verificar antes a quantidade de ZEROS presentes, pois a média pode não representar a realidade. Interessante talvez saber se a maioria é 0, se sim, podemos preencher com zeros)
# todo: somar a quantidade de zeros nessa coluna para avaliar

In [ ]:
# BLOCO Exercise frequency: (é possível fazer uma média entre todos e preencher)

Ainda focando em melhorar o dataframe, vimos que as colunas "Waketime" e "Bedtime" não estão no forma datetime, o que pode ser útil em uma análise futura. Vamos fazer essa conversão.

In [19]:
df['Bedtime'] = pd.to_datetime(df['Bedtime'], errors='coerce')
df['Wakeup time'] = pd.to_datetime(df['Wakeup time'], errors='coerce')

# vamos verificar se o Dtype dessas colunas mudaram de fato

# Selecionando as colunas desejadas
colunas_datatime = ['Bedtime', 'Wakeup time']
df_selecionado = df[colunas_datatime]

# Chamando o método info() no DataFrame selecionado
df_selecionado.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Bedtime      189 non-null    datetime64[ns]
 1   Wakeup time  189 non-null    datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 7.2 KB
